# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120176e+02     1.696095e+00
 * time: 0.8041019439697266
     1     1.105720e+01     1.086551e+00
 * time: 3.409572124481201
     2    -1.125991e+01     1.105889e+00
 * time: 4.099808931350708
     3    -3.357207e+01     8.889625e-01
 * time: 5.11748194694519
     4    -4.722443e+01     6.947145e-01
 * time: 6.098887920379639
     5    -5.665492e+01     2.220052e-01
 * time: 7.121997117996216
     6    -5.965648e+01     1.355719e-01
 * time: 7.8200600147247314
     7    -6.081371e+01     5.478583e-02
 * time: 8.509582042694092
     8    -6.124078e+01     6.818591e-02
 * time: 9.199940919876099
     9    -6.152511e+01     4.439267e-02
 * time: 9.88260006904602
    10    -6.175443e+01     3.595450e-02
 * time: 10.569936037063599
    11    -6.193439e+01     2.871679e-02
 * time: 11.260606050491333
    12    -6.200804e+01     2.462809e-02
 * time: 11.941401958465576
    13    -6.208258e+01     2.052742e-02
 * time: 12.642647981643677
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557689
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666457349
